More details see the post:
https://ricardodeazambuja.com/

# Record audio from your microphone

In [1]:
from samplerate.converters import resample

# Load model

In [2]:
import tensorflow as tf
from tensorflow import keras
import pickle
import cv2
import librosa
import pandas as pd
import soundfile as sf
from tqdm import tqdm
import os

In [3]:
model_fe_128 = keras.models.load_model("model/ResNet152_128")

In [4]:
with open("model/PCA_128.pkl", "rb") as f:
      pca_128 = pickle.load(f)

In [5]:
with open("model/SVM_128.pkl", "rb") as f:
      svm_128 = pickle.load(f)

In [6]:
_model_128 = [model_fe_128,pca_128,svm_128]

# Preprocessing


### Get duration of source audio

In [7]:
from mutagen.mp3 import MP3
from mutagen.wave import WAVE

# function to convert the seconds into readable format
def convert(seconds):
    hours = seconds // 3600
    seconds %= 3600
    mins = seconds // 60
    seconds %= 60
    time_string = f'{hours}h:{mins}m:{seconds}s'
    return hours,mins,seconds,time_string

def get_duration(filename):
    if filename.split('.')[-1] in ['mp3', 'MP3']:
        audio = MP3(filename)
    elif filename.split('.')[-1] in ['wav', 'WAV']:
        audio = WAVE(filename)
    else:
        print('System only accept .wav or .mp3 file')
        return 0
    
    audio_info = audio.info    
    length_in_secs = int(audio_info.length)
    hours, mins, seconds, time_str = convert(length_in_secs)

    src_duration = hours*3600 + mins*60 + seconds
    return src_duration

### Extract subclips from an audio

In [8]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
    
def extract_clips(filename, duration_short):
    if not os.path.exists('subclips'):
        os.mkdir('subclips')
    else:
        clear_file('subclips')
    src_duration = get_duration(filename)
    if src_duration <= duration_short:
        return [filename]
    else:
        files = []
        count = 1
        duration_of_clip = duration_short #in seconds, duration of final audio clip
        src_duration = src_duration #in seconds, the duration of the original audio
        for i in range(0, src_duration +1 , duration_of_clip):
            out_fname = filename.split('.')[0]
            path_name = ''
            path_name = f'{out_fname}_clip{count}.wav'
            path_name=path_name.replace('upload','subclips')
            ffmpeg_extract_subclip(filename, i, i + duration_of_clip,targetname=path_name)
            files.append(path_name)
            count += 1
        return files

### Converte video to audio

In [9]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip, ffmpeg_extract_audio

def get_audio_from_video(filename):
    des_file = filename.split('.')[0]+'.wav'
    ffmpeg_extract_audio(filename,des_file)
    return des_file 

In [10]:
def clear_file(folder):
    for file in os.listdir(folder):
        os.remove(os.path.join(folder,file))

### Convert mp3 -> wav

In [11]:
from os import path
from pydub import AudioSegment

In [12]:
def convert_audio(filename):
    des_file = filename.split('.')[0]+'.wav'
    sound = AudioSegment.from_mp3(filename)
    sound.export(des_file, format="wav")
    return des_file

### ConverteStoHMS

In [13]:
def converte_time(time):
    return

### Post processing

# SER System

In [14]:
import pickle
import numpy as np
import os

#for loading and visualizing audio files
import librosa

#to play audio
import IPython.display as ipd
import cv2
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv

In [15]:
import time

In [16]:
def demo(model_128, duration_short = 10, file_path=0):
    tic=time.perf_counter()
    #model [resnet,pca,svm]
    new_audio,new_samplerate = [],0
    
    # Record audio
    if file_path==0:
        freq = 44100
        sr = freq
        duration = duration_short
        print('Recording. Please talk anything...')
        audio = sd.rec(int(duration * freq), samplerate=freq, channels=2)
        sd.wait()
        print('analyzing emotions in audio...')
        sf.write('input.wav', audio, sr)
        list_audio = ['input.wav']
        
    # Input audio file or video
    else:
        if file_path.split('.')[-1] in ['mp4', 'MP4', 'avi', 'AVI']:  #input video clip
            file_audio = get_audio_from_video(file_path)
        else:
            file_audio = convert_audio(file_path)
        list_audio = extract_clips(file_audio, duration_short)
    
    emotions = []
    times = []
    start_time = 0
    for i in tqdm(range(len(list_audio))):
        file = list_audio[i]
        new_audio ,new_samplerate = librosa.load(file)
        length = 851895
        new_audio = librosa.util.fix_length(new_audio,length)
        
        sample_128 = new_audio
        sr_128 = new_samplerate
        mfcc_128 = librosa.feature.mfcc(sample_128, sr=sr_128,n_mfcc=128)
        mfcc_128_3d = cv2.merge((mfcc_128.T,mfcc_128.T,mfcc_128.T))
        fm_128 = model_128[0].predict(np.array([mfcc_128_3d]))
        fm_pca_128 = model_128[1].transform(fm_128)

        prob = model_128[2].predict_proba(fm_pca_128)[0]
        if prob[1]>0.5:
            emotion = 'NEUTRAL'
        else:
            emotion = 'ANGRY - Probability: {}%'.format(np.round(prob[0]*100,2))
        emotions.append(emotion)
        end_time = start_time + duration_short
        times.append('{} - {}'.format(start_time, end_time))
        start_time = end_time
    toc=time.perf_counter() 
    return emotions, times, list_audio,toc-tic

## Test demo

In [17]:
from IPython.display import clear_output

#emotion, time, list_audio, running_time = demo(_model_128, duration_short=15)
#clear_output()

In [18]:
# import IPython
# import matplotlib.pyplot as plt

# for i in range(len(list_audio)):
#     sub_clip = list_audio[i]
#     emo = emotion[i]
#     IPython.display.display(IPython.display.Markdown(emo))
#     IPython.display.display(IPython.display.Audio(url=sub_clip))

### Concatenate audio

In [19]:
from moviepy.editor import concatenate_audioclips, AudioFileClip

In [20]:
def concatenate_audio_moviepy(audio_clip_paths, output_path):
    clips = [AudioFileClip(c) for c in audio_clip_paths]
    final_clip = concatenate_audioclips(clips)
    final_clip.write_audiofile(output_path)

In [21]:
def find_emotion(emotions):
    index_emotion = []
    name_emotion = []
    emotion = ''
    for i in range(len(emotions)):
        if emotions[i].split(' - ')[0] != emotion:
            index_emotion.append(i)
            name_emotion.append(emotions[i].split(' - ')[0])
            emotion = emotions[i].split(' - ')[0]
    
    for i in range(len(name_emotion)):
        if name_emotion[i] == "ANGRY":
            if i == len(name_emotion)-1:
                temp_e = emotions[index_emotion[i]:]
                probability = []
                for j in range(len(temp_e)):
                    probability.append(float(temp_e[j].split(' - ')[1].split(' ')[1][:-1]))
                name_emotion[i] = name_emotion[i] + f' - Probability: {np.array(probability).mean()}%'
            else:
                temp_e = emotions[index_emotion[i]:index_emotion[i+1]]
                probability = []
                for j in range(len(temp_e)):
                    probability.append(float(temp_e[j].split(' - ')[1].split(' ')[1][:-1]))
                name_emotion[i] = name_emotion[i] + f' - Probability: {np.array(probability).mean()}%'
    
    return index_emotion, name_emotion

In [22]:
def merge_emotion(emotions,times,list_files):
    tic=time.perf_counter()
    index, emotion = find_emotion(emotions)
    list_file_merge = []
    list_times = []
    for i in range(len(index)):
        if i == len(index) - 1:
            list_file = list_files[index[i]:]
            path_name = list_files[0][:-5] + str(i+1) + '.wav'
            path_name=path_name.replace("subclips","result")
            list_file_merge.append(path_name)
            list_times.append(str(convert(int(times[index[i]].split(' - ')[0]))[-1])+" - "+str(convert(int(times[-1].split(' - ')[1]))[-1]))
            concatenate_audio_moviepy(list_file,path_name)
        else:
            list_file = list_files[index[i]:index[i+1]]
            path_name = list_files[0][:-5] + str(i+1) + '.wav'
            path_name=path_name.replace("subclips","result")
            list_file_merge.append(path_name)
            list_times.append(str(convert(int(times[index[i]].split(' - ')[0]))[-1])+" - "+str(convert(int(times[index[i+1]-1].split(' - ')[1]))[-1]))
            concatenate_audio_moviepy(list_file,path_name)
    clear_output()
    toc=time.perf_counter()
    return emotion, list_times,list_file_merge,toc-tic

In [23]:
# emotion_merge, time_merge, audio_merge = merge_emotion(emotion,time,list_audio)


In [24]:
# for i in range(len(audio_merge)):
#     sub_clip = audio_merge[i]
#     emo = emotion_merge[i]
#     IPython.display.display(IPython.display.Markdown(emo))
#     IPython.display.display(IPython.display.Audio(url=sub_clip))

### Deploy 


In [25]:
import json
from flask import Flask, request
from flask import render_template
from flask_cors import CORS, cross_origin

In [26]:
from flask import send_file
import os
import zipfile
import secrets

In [27]:
def create_path(path, TOKEN=None):
    if TOKEN:
        path = os.path.join(path, TOKEN)
    if not os.path.exists(path):
        os.makedirs(path)
    return path
def clear_file(path, TOKEN=None):
    if TOKEN:
        path = os.path.join(path, TOKEN)
    # create_path(path)
    for f in os.listdir(path):
        os.remove(os.path.join(path, f))

def clear_path(path, TOKEN=None):
    if TOKEN:
        path = os.path.join(path, TOKEN)
    try:
        shutil.rmtree(path)
    except:
        None

In [ ]:
app = Flask(__name__)
#app.config['JSON_AS_ASCII'] = False


CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
app.config['RESULT'] = ''
app.config['TOKEN'] = ''
@app.route('/app', methods=['POST'])
@cross_origin(origin='*')
def process():
    TOKEN = secrets.token_urlsafe(50) 
    app.config['TOKEN'] =  TOKEN
    path_token = create_path('client/',TOKEN)
    path_upload = create_path(path_token,'upload') + '/'
    path_result = create_path(path_token,'result') + '/'
    create_path(path_token,'subclips')
    if request.method == 'POST':
        obj = request.form
        x = request.files['audio']
        x.save(path_upload+x.filename)
        emotion, time, list_audio, running_time_1 = demo(_model_128, duration_short=15, file_path = path_upload+x.filename)
        emotion_merge, time_merge, audio_merge, running_time_2 = merge_emotion(emotion,time,list_audio)
        app.config['RESULT'] = path_result
        df = pd.DataFrame({'Emotion':emotion_merge,'Time':  time_merge})
        df.to_csv(path_result+'emotion.csv',index=False)
        
        return {
                'emotion': emotion_merge,
                'time': time_merge,
                'Running time': f"{(running_time_1 + running_time_2)/60:0.4f} minutes",
                'download' : 'http://127.0.0.1:5000/download/' + TOKEN
                }
    else:
        return "Please use POST method"
            

@app.route('/download/<TOKEN>')
def download_all(TOKEN = app.config['TOKEN']):
    path_result_zip = app.config['RESULT'][:-1].replace('result','RESULT.zip').replace("client/","client\\")
    zipf = zipfile.ZipFile(path_result_zip,'w', zipfile.ZIP_DEFLATED)
    for root,dirs, files in os.walk(app.config['RESULT']):
        for file in files:
            zipf.write(app.config['RESULT']+file)
    zipf.close()
    return send_file(path_result_zip,
            mimetype = 'zip',
            attachment_filename= "RESULT.zip",
            as_attachment = True)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [01:32<00:00,  1.38s/it]


MoviePy - Writing audio in client/vSGk9G4egDV1m4K3UvNrS3ncSLgDHS6Golk-RXQkUMXeovmWaADXJDn2p8mRk92WOes\result/tran-my-gap-chm_clip1.wav


...

MoviePy - Done.
MoviePy - Writing audio in client/vSGk9G4egDV1m4K3UvNrS3ncSLgDHS6Golk-RXQkUMXeovmWaADXJDn2p8mRk92WOes\result/tran-my-gap-chm_clip11.wav


MoviePy - Done.
MoviePy - Writing audio in client/vSGk9G4egDV1m4K3UvNrS3ncSLgDHS6Golk-RXQkUMXeovmWaADXJDn2p8mRk92WOes\result/tran-my-gap-chm_clip12.wav


MoviePy - Done.
MoviePy - Writing audio in client/vSGk9G4egDV1m4K3UvNrS3ncSLgDHS6Golk-RXQkUMXeovmWaADXJDn2p8mRk92WOes\result/tran-my-gap-chm_clip13.wav


MoviePy - Done.
MoviePy - Writing audio in client/vSGk9G4egDV1m4K3UvNrS3ncSLgDHS6Golk-RXQkUMXeovmWaADXJDn2p8mRk92WOes\result/tran-my-gap-chm_clip14.wav


MoviePy - Done.
